In [18]:
%pip install "duckdb==1.2.2"
%pip install "fpdf2==2.8.3"
%pip install "plotly>=6,<7"
%pip install -U "kaleido==0.2.1"


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached kaleido-0.2.1-py2.py3-none-win_amd64.whl.metadata (15 kB)
Using cached kaleido-0.2.1-py2.py3-none-win_amd64.whl (65.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import duckdb
import plotly.express as px
from fpdf import FPDF

In [17]:
import fpdf

fpdf.__version__

'2.8.3'

In [5]:
# Create a DuckDB connection and read a CSV file into a DuckDB table
db = duckdb.connect()
db.execute(
    """
    CREATE TABLE transactions AS
    SELECT * FROM read_parquet('https://raw.githubusercontent.com/subwaymatch/mba564b-2025-redesign/main/03-SQL-database-to-report/data/state-of-delaware-pcard-transactions.parquet');
"""
)

In [6]:
db.sql("SELECT * FROM transactions LIMIT 2;")

┌─────────────┬───────────────┬────────────────────┬────────────────────────┬───────────────────────┬────────────────────────────────────────────────────┬─────────────────────┬────────┐
│ fiscal_year │ fiscal_period │     department     │        division        │       merchant        │                category_description                │  transaction_date   │ amount │
│    int64    │     int64     │      varchar       │        varchar         │        varchar        │                      varchar                       │    timestamp_ns     │ double │
├─────────────┼───────────────┼────────────────────┼────────────────────────┼───────────────────────┼────────────────────────────────────────────────────┼─────────────────────┼────────┤
│        2017 │             1 │ LEGISLATIVE BRANCH │ General Assembly House │ VZWRLSS*APOCC VISB    │ Telecom Incl Prepaid-Recurring Phone Svcs          │ 2016-07-14 00:00:00 │  11.65 │
│        2017 │             1 │ LEGISLATIVE BRANCH │ General Assembly 

In [7]:
query_num_rows = db.sql("SELECT COUNT(*) AS num_rows FROM transactions;")
query_num_rows

┌──────────┐
│ num_rows │
│  int64   │
├──────────┤
│  1585108 │
└──────────┘

In [8]:
query_top_10_departments = db.sql(
    """
    SELECT 
    department, 
    SUM(amount)::INT AS total_spent
        FROM transactions
        GROUP BY department
        ORDER BY total_spent DESC
        LIMIT 10;
"""
)

query_top_10_departments

┌────────────────────────────────┬─────────────┐
│           department           │ total_spent │
│            varchar             │    int32    │
├────────────────────────────────┼─────────────┤
│ DEPT OF CORRECTIONS            │   213372349 │
│ DEPT OF TRANSPORTATION         │   105867361 │
│ DEPT OF NATRL RES AND ENV CONT │    40121063 │
│ DEL TECH AND COMM COLLEGE      │    38555315 │
│ EXECUTIVE                      │    28860860 │
│ DEPT OF TECHNOLOGY AND INFOR   │    27838468 │
│ DEPT OF SAFETY AND HOMELAND    │    22436893 │
│ DEPT OF HEALTH AND SOCIAL SV   │    22131629 │
│ DEPT OF STATE                  │    17263785 │
│ APPOQUINIMINK SCHOOL DISTRICT  │    14194091 │
├────────────────────────────────┴─────────────┤
│ 10 rows                            2 columns │
└──────────────────────────────────────────────┘

In [9]:
pdf = FPDF()
pdf.add_page()
pdf.set_font("Helvetica", size=14, style="B")
pdf.cell(text="Top 10 Departments by Spending")

pdf.set_font("Helvetica", size=10)
pdf.ln(10)

with pdf.table(text_align=("LEFT", "RIGHT")) as table:
    header_row = table.row()
    header_row.cell("Department", border="TOP")
    header_row.cell("Total Spent", border="TOP")

    for data_row in query_top_10_departments.fetchall():
        new_row = table.row()

        new_row.cell(data_row[0], border="TOP")
        new_row.cell(f"${data_row[1]:,}", border="TOP")

pdf.output("report.pdf")

In [ ]:
query_department_monthly_spending = db.sql(
    """
    SELECT 
        department,
        strftime(transaction_date, '%Y-%m') AS year_month,
        COUNT(*) AS transaction_count,
        SUM(amount)::INT64 AS total_amount
    FROM transactions
    WHERE department = 'DEPT OF TRANSPORTATION' OR department = 'DEPT OF CORRECTIONS'
    GROUP BY department, year_month
    ORDER BY department, year_month;
"""
)


query_department_monthly_spending

┌────────────────────────┬────────────┬───────────────────┬──────────────┐
│       department       │ year_month │ transaction_count │ total_amount │
│        varchar         │  varchar   │       int64       │    int64     │
├────────────────────────┼────────────┼───────────────────┼──────────────┤
│ DEPT OF CORRECTIONS    │ 2016-06    │               278 │       151435 │
│ DEPT OF CORRECTIONS    │ 2016-07    │              1297 │      2077463 │
│ DEPT OF CORRECTIONS    │ 2016-08    │              1685 │      1369367 │
│ DEPT OF CORRECTIONS    │ 2016-09    │              1602 │      1893165 │
│ DEPT OF CORRECTIONS    │ 2016-10    │              1736 │      1439538 │
│ DEPT OF CORRECTIONS    │ 2016-11    │              1268 │      1811507 │
│ DEPT OF CORRECTIONS    │ 2016-12    │              1496 │      1848660 │
│ DEPT OF CORRECTIONS    │ 2017-01    │              1470 │      1891260 │
│ DEPT OF CORRECTIONS    │ 2017-02    │              1258 │      1182009 │
│ DEPT OF CORRECTIONS    

In [11]:
query_department_monthly_spending.df()

,department,year_month,transaction_count,total_amount
0,DEPT OF CORRECTIONS,2016-06,278,151435
1,DEPT OF CORRECTIONS,2016-07,1297,2077463
2,DEPT OF CORRECTIONS,2016-08,1685,1369367
3,DEPT OF CORRECTIONS,2016-09,1602,1893165
4,DEPT OF CORRECTIONS,2016-10,1736,1439538
...,...,...,...,...
207,DEPT OF TRANSPORTATION,2024-11,1067,700195
208,DEPT OF TRANSPORTATION,2024-12,1125,991898
209,DEPT OF TRANSPORTATION,2025-01,1457,1131349
210,DEPT OF TRANSPORTATION,2025-02,1320,995017


In [ ]:
fig = px.line(
    query_department_monthly_spending.df(),
    x="year_month",
    y="total_amount",
    color="department",
    template="simple_white",
    title="Monthly Spending by Department",
)

fig.update_layout(
    xaxis_title="Year-Month",
    yaxis_title="Total Amount Spent",
    xaxis_tickformat="%Y-%m",
    yaxis_tickprefix="$",
    yaxis_tickformat=",.0f",
    showlegend=False,
)

# fig.write_image("chart.png", width=1000, height=600)

fig.show()

In [13]:
import plotly

plotly.__version__

'5.24.1'